# Grating Coupler Inverse Design

Gradient-based topology optimization of a silicon photonic grating coupler
using the Hyperwave Community SDK.

**Workflow:**
1. Define SOI layer stack and rectangular design region
2. Build 3D structure and visualize permittivity
3. Create Gaussian beam source and waveguide mode
4. Run forward simulation to verify setup
5. Optimize on cloud GPU with adjoint gradients
6. Visualize optimized fields and coupling efficiency

> **Note:** This notebook demonstrates the inverse design workflow as a preview.
> The optimized design is not yet fabrication-ready -- binarization penalties and
> fabrication constraints (minimum feature size enforcement, etch rule checks)
> are still in development. GDS export should only be used after these
> post-processing steps are applied.

In [ ]:
%pip install "hyperwave-community[gds] @ git+https://github.com/spinsphotonics/hyperwave-community.git@feb-inverse-design-notebook" -q

In [ ]:
import hyperwave_community as hwc
from hyperwave_community.sources import mode_converter
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import time

hwc.configure_api(api_key="your-api-key-here")

## 1. Physical Parameters

Standard 220nm SOI at 1550nm with partial-etch grating coupler.


In [ ]:
# Materials
n_si = 3.48
n_sio2 = 1.44
n_clad = 1.44
n_air = 1.0

# Wavelength
wavelength_um = 1.55

# Layer thicknesses (um)
h_dev = 0.220        # total silicon device layer
etch_depth = 0.110   # partial etch depth
h_box = 2.0          # buried oxide
h_clad = 0.78        # SiO2 cladding
h_sub = 0.8          # silicon substrate
h_air = 1.0          # air above cladding
pad = 3.0            # absorber padding (top and bottom)

# Grid
dx = 0.035           # 35nm resolution
domain = 40.0        # um total domain

# Waveguide
wg_width = 0.5       # um
wg_length = 5.0      # um

# Fiber
beam_waist = 5.2     # um (SMF-28 mode field radius at 1550nm)
fiber_angle = 14.5   # degrees from vertical (standard SMF coupling angle)

# Grid dimensions
Lx = int(domain / dx)
Ly = Lx

# Layer thicknesses in pixels
h_p = int(round(pad / dx))
h0 = int(round(h_air / dx))
h1 = int(round(h_clad / dx))
h2 = int(round(etch_depth / dx))
h3 = int(round((h_dev - etch_depth) / dx))
h4 = int(round(h_box / dx))
h5 = int(round(h_sub / dx))
Lz = h_p + h0 + h1 + h2 + h3 + h4 + h5 + h_p

# Key Z positions (from bottom)
z_etch = h_p + h0 + h1
z_slab = z_etch + h2
z_box = z_slab + h3

# Frequency
wl_px = wavelength_um / dx
freq = 2 * np.pi / wl_px
freq_band = (freq, freq, 1)

# Permittivity
eps_si = n_si**2
eps_sio2 = n_sio2**2
eps_clad = n_clad**2
eps_air = n_air**2

# Density filtering
DENSITY_RADIUS = 3
DENSITY_ALPHA = 0.8
DESIGN_LAYER = 3  # etch layer index

print(f"Grid: {Lx} x {Ly} x {Lz} = {Lx * Ly * Lz:,} voxels")
print(f"Layers (px): pad={h_p} air={h0} clad={h1} etch={h2} slab={h3} BOX={h4} sub={h5} pad={h_p}")
print(f"Resolution: {dx * 1000:.0f} nm")
print(f"Fiber angle: {fiber_angle} deg")

## 2. Design Variables (Theta)

`theta` controls the etch layer (top 110nm of 220nm Si):
`theta=1` = unetched silicon, `theta=0` = etched to cladding.

Fixed waveguide on the left, rectangular design region initialized to 0.5.


In [ ]:
# Waveguide in pixels
wg_len = int(round(wg_length / dx))
wg_hw = int(round(wg_width / 2 / dx))

# Rectangular design region
abs_margin = 80
design_region = {
    'x_start': wg_len,
    'x_end': Lx - abs_margin,
    'y_start': abs_margin,
    'y_end': Ly - abs_margin,
}

# Build theta
theta_init = np.zeros((Lx, Ly), dtype=np.float32)
theta_init[:wg_len, Ly // 2 - wg_hw : Ly // 2 + wg_hw] = 1.0
dr = design_region
theta_init[dr['x_start']:dr['x_end'], dr['y_start']:dr['y_end']] = 0.5

# Plot
fig, ax = plt.subplots(figsize=(8, 8))
extent = [0, Lx * dx, 0, Ly * dx]
ax.imshow(theta_init.T, origin='lower', cmap='PuOr', vmin=0, vmax=1, extent=extent)
rect = Rectangle(
    (dr['x_start'] * dx, dr['y_start'] * dx),
    (dr['x_end'] - dr['x_start']) * dx,
    (dr['y_end'] - dr['y_start']) * dx,
    linewidth=2, edgecolor='lime', facecolor='none', linestyle='--',
)
ax.add_patch(rect)
ax.set_xlabel('x (um)')
ax.set_ylabel('y (um)')
ax.set_title('Initial Theta (design region outlined)')
plt.colorbar(ax.images[0], ax=ax, label='theta')
plt.tight_layout()
plt.show()

print(f"Design region: {(dr['x_end'] - dr['x_start']) * dx:.1f} x "
      f"{(dr['y_end'] - dr['y_start']) * dx:.1f} um")


## 3. Layer Stack and Structure

8-layer SOI stack. The etch layer is controlled by theta with density filtering
(`radius=3`, `alpha=0.8`) for minimum feature size control.

Layers (top to bottom): pad (air, 3um) | air (1um) | cladding (SiO2, 0.78um) |
**etch (SiO2/Si, 0.11um)** | slab (Si, 0.11um) | BOX (SiO2, 2um) |
substrate (Si, 0.8um) | pad (Si, 3um)


In [ ]:
# Build 3D structure from initial theta
density_etch = hwc.density(jnp.array(theta_init), radius=DENSITY_RADIUS, alpha=DENSITY_ALPHA)
zero = jnp.zeros((Lx, Ly))
ones = jnp.ones((Lx, Ly))

# hwc.Layer(density_pattern, permittivity_values, layer_thickness)
layers = [
    hwc.Layer(zero,         eps_air,            h_p),  # pad (top)
    hwc.Layer(zero,         eps_air,            h0),   # air
    hwc.Layer(zero,         eps_clad,           h1),   # cladding
    hwc.Layer(density_etch, (eps_clad, eps_si), h2),   # etch (designable)
    hwc.Layer(ones,         (eps_clad, eps_si), h3),   # slab (solid Si)
    hwc.Layer(zero,         eps_sio2,           h4),   # BOX
    hwc.Layer(zero,         eps_si,             h5),   # substrate
    hwc.Layer(zero,         eps_si,             h_p),  # pad (bottom)
]
structure = hwc.create_structure(layers=layers, vertical_radius=0)
eps_3d = np.array(structure.permittivity[0])

# Permittivity: XY, XZ, YZ cross-sections
z_dev = z_etch + h2 // 2  # middle of etch layer
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

im0 = axes[0].imshow(eps_3d[:, :, z_dev].T, origin='lower', cmap='viridis',
                      extent=[0, Lx * dx, 0, Ly * dx])
axes[0].set_xlabel('x (um)')
axes[0].set_ylabel('y (um)')
axes[0].set_title(f'Permittivity XY (z = {z_dev * dx:.2f} um)')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(eps_3d[:, Ly // 2, :].T, origin='lower', cmap='viridis',
                      aspect='auto', extent=[0, Lx * dx, 0, Lz * dx])
axes[1].axhline(z_etch * dx, color='r', ls='--', lw=0.8)
axes[1].axhline(z_slab * dx, color='r', ls='--', lw=0.8)
axes[1].set_xlabel('x (um)')
axes[1].set_ylabel('z (um)')
axes[1].set_title('Permittivity XZ (y = center)')
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(eps_3d[Lx // 2, :, :].T, origin='lower', cmap='viridis',
                      aspect='auto', extent=[0, Ly * dx, 0, Lz * dx])
axes[2].axhline(z_etch * dx, color='r', ls='--', lw=0.8)
axes[2].axhline(z_slab * dx, color='r', ls='--', lw=0.8)
axes[2].set_xlabel('y (um)')
axes[2].set_ylabel('z (um)')
axes[2].set_title('Permittivity YZ (x = center)')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()


## 4. Source

Unidirectional Gaussian beam via the wave equation error method. Propagates
downward (-z) at 14.5 degrees from vertical, simulating fiber illumination from
above at the standard SMF coupling angle. Negative `theta` tilts the beam
toward the waveguide (in the -x direction).

In [ ]:
# Source position: in the air gap, above cladding
source_z = h_p + h0 - 2
grating_x = int((dr['x_start'] + dr['x_end']) / 2)
grating_y = Ly // 2
waist_px = beam_waist / dx

source_field, input_power = hwc.create_gaussian_source(
    sim_shape=(Lx, Ly, Lz),
    frequencies=jnp.array([freq]),
    source_pos=(grating_x, grating_y, source_z),
    waist_radius=waist_px,
    theta=-fiber_angle,  # negative tilts beam toward waveguide (-x direction)
    phi=0.0,             # tilt in XZ plane
    polarization='y',
    x_span=float(Lx),
    y_span=float(Ly),
    max_steps=5000,
    check_every_n=200,
    show_plots=False,
)

source_offset = (grating_x - Lx // 2, grating_y - Ly // 2, source_z)
input_power = float(jnp.mean(input_power))

# Plot source |Ey| and |Hx|
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax, idx, name in [(axes[0], 1, '|Ey|'), (axes[1], 3, '|Hx|')]:
    im = ax.imshow(np.abs(np.array(source_field[0, idx, :, :, 0])).T,
                   origin='lower', cmap='hot', extent=[0, Lx * dx, 0, Ly * dx])
    ax.set_xlabel('x (um)')
    ax.set_ylabel('y (um)')
    ax.set_title(f'Source {name}')
    plt.colorbar(im, ax=ax)
plt.suptitle(f'Gaussian source (waist = {beam_waist} um, tilt = {fiber_angle} deg)', fontsize=13)
plt.tight_layout()
plt.show()

print(f"Source shape: {source_field.shape}")
print(f"Source offset: {source_offset}")
print(f"Input power: {input_power:.6f}")

## 5. Waveguide Mode

Solve for the fundamental TE eigenmode, then use `mode_converter` to get
the full E+H field (needed for mode overlap loss).


In [ ]:
# Solve eigenmode at the waveguide center
mode_x_pos = wg_len // 2

source_mode, offset_mode, mode_info = hwc.create_mode_source(
    structure=structure,
    freq_band=freq_band,
    mode_num=0,
    propagation_axis='x',
    source_position=mode_x_pos,
    perpendicular_bounds=(Ly // 2 - 50, Ly // 2 + 50),
    z_bounds=(z_etch - 10, z_box + 10),
    visualize=False,
)

n_eff = float(mode_info['beta'][0]) / (2 * np.pi / wl_px)

# Convert E-only mode to full E+H via short waveguide propagation
eps_slice = structure.permittivity[:, mode_x_pos, :, :]
mode_E = mode_info['field']  # (1, 3, 1, y_crop, z_crop)

mode_full = mode_converter(
    mode_E_field=mode_E,
    freq_band=freq_band,
    permittivity_slice=eps_slice,
    propagation_axis='x',
    visualize=False,
)

# Plot E and H components
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
mode_E_np = np.array(mode_E)
mode_full_np = np.array(mode_full)

for i, name in enumerate(['Ex', 'Ey', 'Ez']):
    im = axes[0, i].imshow(np.abs(mode_E_np[0, i, 0]).T, origin='lower',
                            cmap='viridis', aspect='auto')
    axes[0, i].set_title(f'|{name}|')
    axes[0, i].set_xlabel('y (px)')
    axes[0, i].set_ylabel('z (px)')
    plt.colorbar(im, ax=axes[0, i])

for i, name in enumerate(['Hx', 'Hy', 'Hz']):
    im = axes[1, i].imshow(np.abs(mode_full_np[0, 3 + i, 0]).T, origin='lower',
                            cmap='viridis', aspect='auto')
    axes[1, i].set_title(f'|{name}|')
    axes[1, i].set_xlabel('y (px)')
    axes[1, i].set_ylabel('z (px)')
    plt.colorbar(im, ax=axes[1, i])

plt.suptitle(f'Waveguide mode (n_eff = {n_eff:.4f})', fontsize=14)
plt.tight_layout()
plt.show()

print(f"n_eff = {n_eff:.4f}")
print(f"Mode solve error: {float(mode_info['error'][0]):.2e}")
print(f"Mode full shape: {mode_full.shape}")


## 6. Forward Simulation (Setup Verification)

Run a forward simulation with the initial design to verify:
- Source illuminates the design region correctly
- Light propagates toward the waveguide
- Layer stack and monitors are correctly placed

This costs 1 GPU simulation but catches setup errors before optimization.


In [ ]:
# Absorber (auto-scaled for resolution)
abs_params = hwc.get_optimized_absorber_params(
    resolution_nm=dx * 1000,
    wavelength_um=wavelength_um,
    structure_dimensions=(Lx, Ly, Lz),
)
abs_widths = abs_params['absorption_widths']
abs_coeff = abs_params['absorber_coeff']

# Set up monitors for field extraction
monitors = hwc.MonitorSet()
output_x = abs_widths[0] + 10

# XY slice at device layer
monitors.add(hwc.Monitor(shape=(Lx, Ly, 1), offset=(0, 0, z_dev)), name='xy_device')
# XZ slice at y=center
monitors.add(hwc.Monitor(shape=(Lx, 1, Lz), offset=(0, Ly // 2, 0)), name='xz_center')
# YZ slice at x=center
monitors.add(hwc.Monitor(shape=(1, Ly, Lz), offset=(Lx // 2, 0, 0)), name='yz_center')
# Waveguide output
monitors.add(hwc.Monitor(shape=(1, Ly, Lz), offset=(output_x, 0, 0)), name='wg_output')

# Run forward simulation
recipe = structure.extract_recipe()
fwd_results = hwc.simulate(
    structure_recipe=recipe,
    source_field=source_field,
    source_offset=source_offset,
    freq_band=freq_band,
    monitors_recipe=monitors.recipe,
    absorption_widths=abs_widths,
    absorption_coeff=abs_coeff,
    gpu_type="B200",
)

print(f"Forward sim complete: {fwd_results['sim_time']:.1f}s GPU time")


In [ ]:
# Extract fields and plot |E|^2 cross-sections
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, name, title, ext, xlabel, ylabel in [
    (axes[0], 'xy_device', f'|E|^2 XY (z = {z_dev * dx:.2f} um)',
     [0, Lx * dx, 0, Ly * dx], 'x (um)', 'y (um)'),
    (axes[1], 'xz_center', '|E|^2 XZ (y = center)',
     [0, Lx * dx, 0, Lz * dx], 'x (um)', 'z (um)'),
    (axes[2], 'yz_center', '|E|^2 YZ (x = center)',
     [0, Ly * dx, 0, Lz * dx], 'y (um)', 'z (um)'),
]:
    field = np.array(fwd_results['monitor_data'][name])
    E2 = np.sum(np.abs(field[0, 0:3])**2, axis=0).squeeze()
    vmax = np.percentile(E2, 95) * 4
    im = ax.imshow(E2.T, origin='lower', cmap='hot', extent=ext,
                   aspect='auto', vmax=vmax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.colorbar(im, ax=ax, extend='max')

plt.suptitle('Forward simulation (initial design)', fontsize=14)
plt.tight_layout()
plt.show()

# Check power at waveguide output
wg_field = np.array(fwd_results['monitor_data']['wg_output'])
S = hwc.S_from_slice(jnp.mean(jnp.array(wg_field), axis=2))
power = float(jnp.abs(jnp.sum(S[:, 0, :, :], axis=(1, 2))))
print(f"Waveguide output power: {power:.6f}")
print(f"Coupling (approx): {power / input_power * 100:.1f}%")


## 7. Optimization Setup

**Why `structure_spec` instead of `Structure`?** For forward simulation (section 6),
we passed a pre-built `Structure` with the permittivity already computed. But for
inverse design, the cloud GPU needs to **rebuild** the permittivity from a new
`theta` at every optimization step -- it runs the full chain
`theta -> density_filter -> permittivity -> FDTD -> loss` and differentiates through
all of it. So it needs the layer stack template (materials, thicknesses) plus
the density filter parameters (`radius`, `alpha`) to know how to reconstruct the
permittivity from any given theta.

`run_optimization()` sends all parameters in a single request. The GPU runs the
full optimization loop (forward + adjoint FDTD, gradient computation, and Adam
updates per step) and returns results after each completed step. If you interrupt
the kernel, the GPU task is cancelled -- you are only charged for steps that
actually ran.

In [ ]:
def make_structure_spec(layers, design_layer, density_radius, density_alpha,
                        vertical_radius=0):
    """Build structure_spec dict from a layers list for inverse design."""
    return {
        'layers_info': [
            {
                'permittivity_values': (
                    list(l.permittivity_values) if isinstance(l.permittivity_values, tuple)
                    else float(l.permittivity_values)
                ),
                'layer_thickness': int(l.layer_thickness),
                'density_radius': density_radius if i == design_layer else 0,
                'density_alpha': density_alpha if i == design_layer else 0,
            }
            for i, l in enumerate(layers)
        ],
        'construction_params': {'vertical_radius': vertical_radius},
    }

structure_spec = make_structure_spec(layers, DESIGN_LAYER, DENSITY_RADIUS, DENSITY_ALPHA)

# Monitors for gradient computation
loss_monitor_shape = (1, Ly, Lz)
loss_monitor_offset = (output_x, 0, 0)
design_monitor_shape = (Lx, Ly, h2)
design_monitor_offset = (0, 0, z_etch)

# Waveguide mask: force theta = 1 in the waveguide region
waveguide_mask = np.zeros((Lx, Ly), dtype=np.float32)
waveguide_mask[:wg_len, Ly // 2 - wg_hw : Ly // 2 + wg_hw] = 1.0

# Optimization parameters
NUM_STEPS = 5          # increase to 50-100 for production
LR = 0.01

print(f"Optimizer: Adam + cosine decay (LR={LR}, {NUM_STEPS} steps)")
print(f"Loss monitor at x={output_x} ({output_x * dx:.1f} um)")
print(f"Design monitor: {design_monitor_shape} at z={z_etch}")

## 8. Optimization Loop

`run_optimization()` runs the full loop on a cloud GPU and returns results
after each step. Interrupting the kernel cancels the GPU task
immediately -- you are only charged for completed steps.

In [ ]:
results = []

print(f"Running {NUM_STEPS}-step optimization on cloud GPU...")
print(f"(Interrupt kernel to stop early -- GPU task will be cancelled)\n")

try:
    for step_result in hwc.run_optimization(
        theta=theta_init,
        source_field=source_field,
        source_offset=source_offset,
        freq_band=freq_band,
        structure_spec=structure_spec,
        loss_monitor_shape=loss_monitor_shape,
        loss_monitor_offset=loss_monitor_offset,
        design_monitor_shape=design_monitor_shape,
        design_monitor_offset=design_monitor_offset,
        power_axis=0,
        power_maximize=True,
        num_steps=NUM_STEPS,
        learning_rate=LR,
        waveguide_mask=waveguide_mask,
        absorption_widths=abs_widths,
        absorption_coeff=abs_coeff,
        gpu_type="B200",
    ):
        results.append(step_result)
        step = step_result['step']
        loss = step_result['loss']
        grad_max = step_result['grad_max']
        dt = step_result['step_time']
        print(f"Step {step:3d}/{NUM_STEPS}:  loss = {loss:.6f}  "
              f"|grad|_max = {grad_max:.3e}  ({dt:.1f}s)")

except KeyboardInterrupt:
    print(f"\nStopped early. GPU task cancelled.")
    print(f"Results from {len(results)} completed steps saved.")

if results:
    losses = [r['loss'] for r in results]
    best_idx = int(np.argmin(losses))
    print(f"\nBest loss: {losses[best_idx]:.6f} (step {best_idx + 1})")
    theta = results[-1]['theta']
else:
    print("No steps completed.")
    theta = theta_init

## 9. Results


In [ ]:
losses = [r['loss'] for r in results]
best_idx = int(np.argmin(losses))
best_theta = results[best_idx]['theta']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss curve
axes[0].plot(range(1, len(losses) + 1), losses, 'b-o', markersize=3)
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Loss')
axes[0].set_title('Optimization Loss')
axes[0].grid(True, alpha=0.3)

# Initial vs best theta
extent = [0, Lx * dx, 0, Ly * dx]
for ax, th, title in [(axes[1], theta_init, 'Initial'),
                       (axes[2], best_theta, f'Best (step {best_idx + 1})')]:
    im = ax.imshow(th.T, origin='lower', cmap='PuOr', vmin=0, vmax=1, extent=extent)
    ax.set_xlabel('x (um)')
    ax.set_ylabel('y (um)')
    ax.set_title(title)
    plt.colorbar(im, ax=ax)
    rect = Rectangle(
        (dr['x_start'] * dx, dr['y_start'] * dx),
        (dr['x_end'] - dr['x_start']) * dx,
        (dr['y_end'] - dr['y_start']) * dx,
        linewidth=1.5, edgecolor='lime', facecolor='none', linestyle='--',
    )
    ax.add_patch(rect)

plt.tight_layout()
plt.show()

## 10. Optimized Design Field Plots

Run forward simulation with the best design to visualize field distribution
and verify coupling into the waveguide.


In [ ]:
# Build structure with best theta
density_best = hwc.density(jnp.array(best_theta), radius=DENSITY_RADIUS, alpha=DENSITY_ALPHA)

layers_best = [
    hwc.Layer(zero,         eps_air,            h_p),
    hwc.Layer(zero,         eps_air,            h0),
    hwc.Layer(zero,         eps_clad,           h1),
    hwc.Layer(density_best, (eps_clad, eps_si), h2),
    hwc.Layer(ones,         (eps_clad, eps_si), h3),
    hwc.Layer(zero,         eps_sio2,           h4),
    hwc.Layer(zero,         eps_si,             h5),
    hwc.Layer(zero,         eps_si,             h_p),
]
structure_best = hwc.create_structure(layers=layers_best, vertical_radius=0)

# Forward simulation with same monitors
recipe_best = structure_best.extract_recipe()
opt_results = hwc.simulate(
    structure_recipe=recipe_best,
    source_field=source_field,
    source_offset=source_offset,
    freq_band=freq_band,
    monitors_recipe=monitors.recipe,
    absorption_widths=abs_widths,
    absorption_coeff=abs_coeff,
    gpu_type="B200",
)

# Plot |E|^2 cross-sections
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Row 0: |E|^2 XY, XZ, YZ
for ax, name, title, ext, xlabel, ylabel in [
    (axes[0, 0], 'xy_device', f'|E|^2 XY (z = {z_dev * dx:.2f} um)',
     [0, Lx * dx, 0, Ly * dx], 'x (um)', 'y (um)'),
    (axes[0, 1], 'xz_center', '|E|^2 XZ (y = center)',
     [0, Lx * dx, 0, Lz * dx], 'x (um)', 'z (um)'),
    (axes[0, 2], 'yz_center', '|E|^2 YZ (x = center)',
     [0, Ly * dx, 0, Lz * dx], 'y (um)', 'z (um)'),
]:
    field = np.array(opt_results['monitor_data'][name])
    E2 = np.sum(np.abs(field[0, 0:3])**2, axis=0).squeeze()
    vmax = np.percentile(E2, 95) * 4
    im = ax.imshow(E2.T, origin='lower', cmap='hot', extent=ext,
                   aspect='auto', vmax=vmax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.colorbar(im, ax=ax, extend='max')

# Row 1: Individual E-field components at device layer
field_xy = np.array(opt_results['monitor_data']['xy_device'])
for i, name in enumerate(['|Ex|', '|Ey|', '|Ez|']):
    comp = np.abs(field_xy[0, i, :, :, 0])
    im = axes[1, i].imshow(comp.T, origin='lower', cmap='hot',
                            extent=[0, Lx * dx, 0, Ly * dx])
    axes[1, i].set_xlabel('x (um)')
    axes[1, i].set_ylabel('y (um)')
    axes[1, i].set_title(name)
    plt.colorbar(im, ax=axes[1, i])

plt.suptitle('Optimized design fields', fontsize=14)
plt.tight_layout()
plt.show()

# Coupling efficiency
wg_field = np.array(opt_results['monitor_data']['wg_output'])
S = hwc.S_from_slice(jnp.mean(jnp.array(wg_field), axis=2))
power = float(jnp.abs(jnp.sum(S[:, 0, :, :], axis=(1, 2))))
eff = power / input_power * 100
print(f"Waveguide output power: {power:.6f}")
print(f"Coupling efficiency: {eff:.1f}%")
print(f"Loss: {-10 * np.log10(eff / 100):.2f} dB")


## Next Steps

**This is a preview of the inverse design workflow.** The optimized design
contains grayscale (intermediate) density values and has not been processed
for fabrication. Before manufacturing, the following steps are needed:

- **Binarization:** Increase `density_alpha` gradually from 0 to 1 over the
  course of optimization. At `alpha=0` the design is fully freeform (grayscale
  allowed); at `alpha=1` it is fully binarized (every pixel pushed to 0 or 1).
  A good strategy: run 50 steps with `alpha=0` to find a good design, then
  continue for 50 more steps while linearly ramping alpha to 1.
- **Fabrication constraints:** Enforce minimum feature size, minimum spacing,
  and etch design rules for the target foundry process (in development).
- **GDS export:** Only after binarization and constraint enforcement, convert
  to GDSII using `hwc.generate_gds_from_density()`.

**Improving results:**
- **More steps:** Increase `NUM_STEPS` to 50-100. Grating couplers typically
  need 50+ steps to converge; gains slow down but don't fully plateau until
  100-200 steps.
- **Different starting points:** Adjoint optimization uses gradient descent,
  which converges to a local optimum -- not necessarily the global best. The
  loss landscape for photonic inverse design is non-convex, so different
  initial theta values can lead to different final designs. Try several random
  initializations (e.g., `theta_init = 0.3`, `0.5`, `0.7`, or random patterns)
  and keep the best result.
- **Finer grid resolution:** Decrease `dx` (e.g., 25nm or 20nm) for more
  accurate results. Finer grids resolve sub-wavelength features better but
  increase simulation cost quadratically.
- **Multi-frequency optimization:** Expand `freq_band` to multiple wavelengths
  (e.g., 1530-1570nm) to optimize for bandwidth instead of a single wavelength.

**Design parameters:**
- `density_radius` controls minimum feature size via a conical blur filter.
  `radius=3` at 35nm grid enforces roughly 100nm minimum features.
- `density_alpha` controls binarization strength. Start at 0 for exploration,
  increase toward 1 to push the design to fabricable binary values.
- The optimizer (Adam with cosine learning rate decay and gradient clipping) is
  built into `run_optimization` and does not need manual tuning for most cases.